In [ ]:
#uploading dataset, the uploaded file will be available in the current directory.
from google.colab import files
uploaded = files.upload()

Saving vuln_data.sql to vuln_data.sql


In [ ]:
# The configs for using API
!pip install pyngrok

In [ ]:
!pip install flask flask-ngrok

In [ ]:
# The configs for reading data
!apt-get update
!apt-get install postgresql postgresql-contrib

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,517 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,226 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,448 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.ne

In [ ]:
!service postgresql start

 * Starting PostgreSQL 14 database server
   ...done.


In [ ]:
!pip install psycopg2

In [ ]:
#Creating new database for importing the dumped file
!sudo -u postgres psql -c "CREATE DATABASE mydb;"

CREATE DATABASE


In [ ]:
!sudo -u postgres psql mydb < vuln_data.sql

SET
SET
SET
ERROR:  unrecognized configuration parameter "transaction_timeout"
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
CREATE TABLE
ALTER TABLE
CREATE SEQUENCE
ALTER SEQUENCE
ALTER SEQUENCE
ALTER TABLE
COPY 42
 setval 
--------
    801
(1 row)

ALTER TABLE


In [ ]:
#Setting new password
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'my_secure_password';"

ALTER ROLE


In [ ]:
#Ngrok Auth token for API part
from pyngrok import ngrok
ngrok.set_auth_token("")

In [ ]:
#main code
#importing libraries
from flask import Flask, jsonify
from flask_ngrok import run_with_ngrok
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import psycopg2
import json

#PostgreSQL database
def fetch_vulnerabilities_from_db():


  #Connect to the PostgreSQL database
  conn = psycopg2.connect(
    dbname="mydb",
    user="postgres",
    password="my_secure_password",  # new password
    host="localhost"
    )
  cursor = conn.cursor()

  #Query the data
  cursor.execute("SELECT * FROM public.vuln;")
  rows = cursor.fetchall()
  vulnerabilities = []
  for row in rows:

    vulnerability = {
            "id": row[0],
            "title": row[1],
            "description": row[2],
            "severity": row[3],
            "cve": row[4],
            "tool": row[5],
            "endpoint": row[6]
        }
    vulnerabilities.append(vulnerability)

  return vulnerabilities


#BERT(using for text similarity detection)
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

#text to embedding function
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    #using cls of last hidden state as sentence embedding
    return outputs.last_hidden_state[:, 0, :].squeeze(0).numpy()

#vulvulnerability groups
def group_vulnerabilities(vulnerabilities):
    #embedding vectors
    embeddings = []
    for vuln in vulnerabilities:
        text = f"{vuln['title']} {vuln['description']}" #concatenating title and description as a one text to measure the similarity
        embedding = get_embedding(text)
        embeddings.append(embedding)

    #calculating the similarities and dividing into groups
    groups = {}
    group_id = 1
    assigned = [False] * len(vulnerabilities)

    for i, vuln in enumerate(vulnerabilities):
        if assigned[i]:
            continue

        groups[group_id] = [vuln]
        assigned[i] = True

        for j in range(i + 1, len(vulnerabilities)):
            if vulnerabilities[i]['endpoint'] == vulnerabilities[j]['endpoint']:
                similarity = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
                if similarity > 0.8 or vulnerabilities[i]['cve'] == vulnerabilities[j]['cve']:  # similarity threshhold and equal cve
                    groups[group_id].append(vulnerabilities[j])
                    assigned[j] = True

        group_id += 1

    #labeling
    result = []
    for gid, group in groups.items():
        for item in group:
            item['tag'] = f"group_{gid}"
            result.append(item)

    return result



#Flask app
app = Flask(__name__)
run_with_ngrok(app)  #Initialize ngrok


@app.route('/')
def index():
    return jsonify({"message": "Welcome to the Vulnerabilities API!"}) #the initial page message

@app.route('/vulnerabilities')
def get_vulnerabilities():
    vulnerabilities = fetch_vulnerabilities_from_db()  #fetching vulnerabilities from database
    data = group_vulnerabilities(vulnerabilities)
    return jsonify(data) #json format

if __name__ == '__main__':
    # Start ngrok tunnel
    public_url = ngrok.connect(5000)
    print("Public URL:", public_url)
    app.run()


    {
        "id": 773,
        "title": "Information disclosure in config",
        "description": "Application config files are publicly accessible, causing information disclosure.",
        "severity": "low",
        "cve": "null",
        "tool": "ToolA",
        "endpoint": "/config",
        "tag": "group_1"
    },
    {
        "id": 763,
        "title": "Config information disclosure",
        "description": "Leaking configuration data at /config can give attackers insights into the system.",
        "severity": "medium",
        "cve": "null",
        "tool": "ToolC",
        "endpoint": "/config",
        "tag": "group_1"
    },
    {
        "id": 772,
        "title": "Remote Code Execution at /config",
        "description": "Attackers can achieve code execution on the server via /config if not patched.",
        "severity": "high",
        "cve": "null",
        "tool": "ToolD",
        "endpoint": "/config",
        "tag": "group_1"
    },
    {
        "id": 776,
    